In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [2]:
import urllib.request

url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py"
urllib.request.urlretrieve(url, "minsearch.py")

('minsearch.py', <http.client.HTTPMessage at 0x7a6e3560df10>)

In [3]:
import minsearch

In [4]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
client = OpenAI()

In [13]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'It depends on the specific course and the institution offering it. Some courses allow late enrollment, especially if they are designed to be flexible or self-paced. Here are a few steps you can take:\n\n1. **Contact the Instructor or Program Coordinator**: Reach out to the course instructor or the program coordinator to inquire about the possibility of late enrollment.\n\n2. **Check the Institution’s Policies**: Look into the institution’s policies on enrollment deadlines. Some schools have specific periods during which late registration is allowed.\n\n3. **Consider the Impact**: If you’re permitted to enroll late, consider how missing the initial classes might affect your ability to catch up and succeed in the course.\n\n4. **Explore Online Options**: Some online courses offer rolling admissions, allowing you to start anytime. Explore if there’s an online version of the course that suits your needs.\n\n5. **Audit the Course**: If formal enrollment is not possible, ask if you can audi

In [14]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [18]:
rag(query)

'To run Kafka in a Java environment, navigate to the project directory and use the following command in the terminal:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'

In [19]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. You are also eligible to submit the homework assignments. Keep in mind that there will be deadlines for submitting the final projects, so it's best not to leave everything until the last minute."

In [20]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [21]:
from elasticsearch import Elasticsearch

In [22]:
#es_client = Elasticsearch('https://localhost:9200', verify_certs=False)
es_client = Elasticsearch('http://localhost:9200') 

#from elasticsearch import Elasticsearch

#es_client = Elasticsearch(
#    "https://localhost:9200",
#    basic_auth=("elastic", "yourpassword"),
#    verify_certs=False  # Only use verify_certs=False for local/testing/self-signed certs
#)

In [23]:
import requests
print(requests.get("http://localhost:9200").text)
#print(requests.get("http://localhost:9200/_cat/indices?v").text)


{
  "name" : "8ca990e68ffd",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "7J_uVbXcTr-RpFE_facjxw",
  "version" : {
    "number" : "8.18.1",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "df116ec6455476a07daafc3ded80e2bb1a3385ed",
    "build_date" : "2025-04-30T10:07:44.026929518Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.1",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}



In [24]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)


ConnectionTimeout: Connection timed out

In [25]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

#es_client = Elasticsearch("http://localhost:9200", request_timeout=30)
es_client.index(index=index_name, document=doc)

ConnectionTimeout: Connection timed out

In [ ]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [26]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:10<?, ?it/s]


ConnectionTimeout: Connection timed out

In [ ]:
query = 'I just disovered the course. Can I still join it?'

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
rag(query)

BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Content-Type, Accept]', Accept version must be either version 8 or 7, but found 9. Accept=application/vnd.elasticsearch+json; compatible-with=9)